# policy
> Agent | Policy
- order: 5

In [ ]:
#| default_exp policy

In [ ]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections import namedtuple, defaultdict
import numpy as np
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
from copy import copy
# import time
# import math
# from pprint import pprint
## !pip install -U "ray"
import ray
# import json
import random

from fastcore.basics import patch
# from AIShiftScheduler_KE5.config import *
import AIShiftScheduler_KE5.config as cf
# from AIShiftScheduler_KE5.model import *


len(RESOURCE_TYPES)=3
['Manager', 'AssistMngr', 'RetailAssoc']

len(TYPES)=13
['Manager', 'AssistMngr', 'AssistMngr', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc']
len(aNAMES)=13
['Manager_Matt', 'AssistMngr_Mike', 'AssistMngr_Tanner', 'RetailAssoc_Jake', 'RetailAssoc_James', 'RetailAssoc_Jane', 'RetailAssoc_John', 'RetailAssoc_Jim', 'RetailAssoc_Jenny', 'RetailAssoc_Jeremy', 'RetailAssoc_Judy', 'RetailAssoc_Julie', 'RetailAssoc_Jeffrey']

len(bNAMES)=3
['Manager', 'AssistMngr', 'RetailAssoc']

len(abNAMES)=13
['Manager_Matt___Manager', 'AssistMngr_Mike___AssistMngr', 'AssistMngr_Tanner___AssistMngr', 'RetailAssoc_Jake___RetailAssoc', 'RetailAssoc_James___RetailAssoc', 'RetailAssoc_Jane___RetailAssoc', 'RetailAssoc_John___RetailAssoc', 'RetailAssoc_Jim___RetailAssoc', 'RetailAssoc_Jenny___RetailAssoc', 'RetailAssoc_Jeremy___RetailAssoc', 'RetailAssoc_Judy___RetailAssoc', 'RetailAsso

In [ ]:
pd.__version__

'2.0.3'

In [ ]:
# !pip freeze | grep pandas
# !pip freeze|grep openpyxl
# !pip freeze

In [ ]:
# !pip install -U ray==2.8.1
# !pip install openpyxl

In [ ]:
#| export
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#| export
@ray.remote
def parallel_run_grid_sample_paths(pol, theta, piName, L, T):
# def parallel_run_grid_sample_paths(pol, theta, piName):    
    CcumIomega__lI = []
    record = []
    for l in range(1, L + 1): ## for each sample-path
        print(f'\t%%% {l=}')
        from AIShiftScheduler_KE5.model import Model ## here, else circular import for Model & Policy
        M = Model()
        pol.model = M
        record_l = [piName, theta, l]
        dt = pd.to_datetime(cf.START_DATE_TIME)
        dt_delta = pd.Timedelta(cf.DATE_TIME_DELTA)
        for t in range(T): ## for each transition/step
            ## print(f'\t%%% {t=}')
            ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            ## Lookup new (today's) decision
            getattr(pol, piName)(t, dt, pol.model.S_t, pol.model.x_t, theta)

            ## sit in post-decision state until end of cycle (evening),
            ## waiting for exog info to arrive

            ## Change from today to tomorrow
            ## S_t, Ccum, x_t = self.model.step(t, x_t, theta)
            record_t = pol.model.step(t, dt, theta)
            ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            record.append(record_l + record_t)
            dt = dt + dt_delta
        CcumIomega__lI.append(pol.model.Ccum) ##just above (SDAM-eq2.9)
    return (CcumIomega__lI, record)

In [ ]:
class MyClass(int): pass

In [ ]:
@patch
def func(self: MyClass, a): return self + a

In [ ]:
mc = MyClass(3)

In [ ]:
mc.func(10)

13

In [ ]:
#| export
class Policy(): pass

In [ ]:
#| export
@patch
def __init__(self:Policy, model):
    self.model = model
    self.Policy = namedtuple('Policy', cf.piNAMES) ## 'class'
    self.Theta = namedtuple('Theta', cf.thNAMES) ## 'class'

In [ ]:
#| export
@patch
def build_policy(self:Policy, info):
    return self.Policy(*[info[pin] for pin in cf.piNAMES])

In [ ]:
#| export
@patch
def build_theta(self:Policy, info):
    return self.Theta(*[info[thn] for thn in cf.thNAMES])

In [ ]:
#| export
@patch
def X__Alloc(self:Policy, t, dt, S_t, x_t, theta):
    ## print(f"\n..... Policy.X__Alloc() .....\n{t=}, {dt=}")
    demandsToService = []
    for rt in cf.RESOURCE_TYPES:
      number = S_t['D_t'].loc[
        S_t['D_t']['Type']==rt,
        ['DMax_t']
      ].squeeze()
      demandsToService.append((rt, round(number))) ##whole number of resources
    ## print(f"demandsToService:\n{demandsToService}")
    for demand in demandsToService:
      resourceType, number = demand; ##print(f'{resourceType=}, {number=}')
      candidates_avail = S_t['R_t'].loc[
        (S_t['R_t']['Type'] == resourceType) & \
        (S_t['R_t']['RAvail_t'] == 1) & \
        (S_t['R_t']['RComplete_t'] == 0) & \
        (S_t['R_t']['RCumSlots_t'] < cf.MAX_DAILY_SLOT_RUN),
        ## (S_t['R_t']['RUtil_t'] < cf.GOV_UTIL_THRESH) & \
        ['ResourceId', 'Type', 'RAvail_t', 'RComplete_t', 'RCumSlots_t', 'RCumMerits_t']
      ]; ## print(f'candidates_avail BEFORE arrangement for selection=\n{candidates_avail}')
      ## merge with previous allocations to enhance contiguous allocs
      candidates = \
        S_t['xAlloc_t_1']\
        .merge(candidates_avail, left_index=True, right_index=True)\
        .sort_values(by='Allocd_t', ascending=False)
      candidates.rename(columns={'Allocd_t': 'Allocd_t_1'}, inplace=True)
      if len(candidates) > 0 and number > 0:
        if theta.thSelect == 'random':
          candidates = candidates.sample(frac = 1) ## shuffle rows; now opt/non have different x's
          candidates = candidates.sort_values(by=['Allocd_t_1'], ascending=False)
        elif theta.thSelect == 'ranked_CumMerits':
          candidates = candidates.sort_values(by=['Allocd_t_1','RCumMerits_t'], ascending=False)
        else:
          print(f'ERROR: Invalid value for theta.thSelect: {theta.thSelect}')
        ## print(f'candidates AFTER arrangement for selection=\n{candidates}')
        if len(candidates) >= number:
          allocs = candidates.iloc[0:number, :]; ##print(f'allocs=\n{allocs}') #pick first 'number' avails
        elif len(candidates) < number:
          ##print('%%% NOT ENOUGH candidates')
          allocs = copy(candidates); ##print(f'allocs=\n{allocs}') ##pick all candidates
          unallocated_demands = number - len(candidates) ##unmet demands
          ## self.model.Ccum -= unallocated_demands
          self.model.Ucum_Total += unallocated_demands
          self.model.Ucum[resourceType] += unallocated_demands
        if len(allocs) == 0:
          x_t['xAlloc_t'].loc[
            x_t['xAlloc_t']['Comb'].map(lambda e: e.split('_')[0]==resourceType),
            ['Allocd_t']
          ] = False
        else:
          x_t['xAlloc_t'].loc[ ##remove all allocs before adding new allocs
            x_t['xAlloc_t']['Comb'].map(lambda e: e.split('_')[0]==resourceType),
            ['Allocd_t']
          ] = False
          x_t['xAlloc_t'].loc[
            x_t['xAlloc_t']['Comb'].apply(lambda x: x.split("_")[1]).isin(allocs['ResourceId']),
            ['Allocd_t']
          ] = True
          ## TODO: update utilizations
        self.model.update_Ccum(t, dt, S_t, x_t, theta)
      else:
        ## print(f'%%% Shift has no resource for demand {demand}, or demand is 0.')
        x_t['xAlloc_t'].loc[
          x_t['xAlloc_t']['Comb'].map(lambda e: e.split('_')[0]==resourceType),
          ['Allocd_t']
        ] = False
      ## print(f"x_t['xAlloc_t']:\n{x_t['xAlloc_t']}")

In [ ]:
#| export
@patch
def run_grid_sample_paths(self:Policy, theta, piName, record, L, T):
# def run_grid_sample_paths(self:Policy, theta, piName, record): !!!!!!!!!!!!!!!!!!!!!!!    
    CcumIomega__lI = []
    for l in range(1, L + 1): ## for each sample-path
      print(f'\t%%% {l=}')
      from AIShiftScheduler_KE5.model import Model ## here, else circular import for Model & Policy
      M = Model()
      ## P = Policy(M) ## SG, NO!, overwrite existing global P
      self.model = M
      record_l = [piName, theta, l]
      dt = pd.to_datetime(cf.START_DATE_TIME)
      dt_delta = pd.Timedelta(cf.DATE_TIME_DELTA)
      for t in range(T): ## for each transition/step
        ## print(f'\t%%% {t=}')
        ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
        ## Lookup new (today's) decision
        getattr(self, piName)(t, dt, self.model.S_t, self.model.x_t, theta)

        ## sit in post-decision state until end of cycle (evening),
        ## waiting for exog info to arrive

        ## Change from today to tomorrow
        ## S_t, Ccum, x_t = self.model.step(t, x_t, theta)
        record_t = self.model.step(t, dt, theta)
        ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
        record.append(record_l + record_t)
        dt = dt + dt_delta
      CcumIomega__lI.append(self.model.Ccum) ##just above (SDAM-eq2.9)
    return CcumIomega__lI

In [ ]:
#| export
@patch
def perform_grid_search_sample_paths(self:Policy, piName, thetas, L, T):
# def perform_grid_search_sample_paths(self:Policy, piName, thetas): !!!!!!!!!!!!!!!!!!!    
    Cbarcum = defaultdict(float)
    Ctilcum = defaultdict(float)
    expCbarcum = defaultdict(float)
    expCtilcum = defaultdict(float)
    numThetas = len(thetas)
    record = []
    print(f'{numThetas=:,}')
    nth = 1
    i = 0; print(f'... printing every {nth}th theta (if considered) ...')
    for theta in thetas:
      if True: ##in case relationships between thetas can be exploited
        ## a dict cannot be used as a key, so we define theta_key, e.g.
        ## theta_key = ((168.0, 72.0), (200.0, 90.0)):
        ## theta_key = tuple(tuple(itm.values()) for itm in theta)
        theta_key = theta ##if theta is not a dict

        ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
        # CcumIomega__lI = self.run_grid_sample_paths(theta, piName, record) !!!!!!!!!!!!!!!
        CcumIomega__lI = self.run_grid_sample_paths(theta, piName, record, L, T)  
        ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        Cbarcum_tmp = np.array(CcumIomega__lI).mean() #(SDAM-eq2.9)
        Ctilcum_tmp = np.sum(np.square(np.array(CcumIomega__lI) - Cbarcum_tmp))/(L - 1)

        Cbarcum[theta_key] = Cbarcum_tmp
        Ctilcum[theta_key] = np.sqrt(Ctilcum_tmp/L)

        expCbarcum_tmp = pd.Series(CcumIomega__lI).expanding().mean()
        expCbarcum[theta_key] = expCbarcum_tmp

        expCtilcum_tmp = pd.Series(CcumIomega__lI).expanding().std()
        expCtilcum[theta_key] = expCtilcum_tmp
        if i%nth == 0: print(f'{i:,}/{(numThetas-1):,}, {Cbarcum[theta_key]:,.0f}, {theta}')
        i += 1
      ##endif
    best_theta = max(Cbarcum, key=Cbarcum.get)
    worst_theta = min(Cbarcum, key=Cbarcum.get)

    best_Cbarcum = Cbarcum[best_theta]
    best_Ctilcum = Ctilcum[best_theta]

    worst_Cbarcum = Cbarcum[worst_theta]
    worst_Ctilcum = Ctilcum[worst_theta]

    thetaStar_expCbarcum = expCbarcum[best_theta]
    thetaStar_expCtilcum = expCtilcum[best_theta]
    thetaStar_expCtilcum[0] = 0 ##set NaN to 0

    ## best_theta_w_names = tuple((
    #   ({
    #     a1NAMES[0]: subvec[0],
    #     a1NAMES[1]: subvec[1]
    #   })) for subvec in best_theta)
    ## best_theta_n = self.build_theta({'thAdj': best_theta_w_names[0]})
    ## best_theta_n = self.build_theta({'thAdj1': best_theta_w_names[0], 'thAdj3': best_theta_w_names[1]})
    ## print(f'best_theta_n:\n{best_theta_n}\n{best_Cbarcum=:.2f}\n{best_Ctilcum=:.2f}')

    ## worst_theta_w_names = tuple((
    #   ({
    #     a1NAMES[0]: subvec[0],
    #     a1NAMES[1]: subvec[1]})) for subvec in worst_theta)
    ## worst_theta_n = self.build_theta({'thAdj': worst_theta_w_names[0]})
    ## worst_theta_n = self.build_theta({'thAdj1': worst_theta_w_names[0], 'thAdj3': worst_theta_w_names[1]})
    ## print(f'worst_theta_n:\n{worst_theta_n}\n{worst_Cbarcum=:.2f}\n{worst_Ctilcum=:.2f}')

    return \
      thetaStar_expCbarcum, thetaStar_expCtilcum, \
      Cbarcum, Ctilcum, \
      best_theta, worst_theta, \
      best_Cbarcum, worst_Cbarcum, \
      best_Ctilcum, worst_Ctilcum, \
      record

In [ ]:
## def parallel_run_grid_sample_paths(...) is a function above this class

In [ ]:
#| export
@patch
def parallel_perform_grid_search_sample_paths(self:Policy, piName, thetas, L, T):
# def parallel_perform_grid_search_sample_paths(self:Policy, piName, thetas):    
      Cbarcum = defaultdict(float)
      Ctilcum = defaultdict(float)
      expCbarcum = defaultdict(float)
      expCtilcum = defaultdict(float)
      numThetas = len(thetas)
      record = []
      print(f'{numThetas=:,}')
      nth = 1
      i = 0; print(f'... printing every {nth}th theta (if considered) ...')
      
      ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
      ## THIS BREAKS:
      ## futures = [self.parallel_run_grid_sample_paths.remote(theta, piName) for theta in thetas]
      # futures = [parallel_run_grid_sample_paths.remote(self, theta, piName) for theta in thetas] !!!!!!!
      futures = [parallel_run_grid_sample_paths.remote(self, theta, piName, L, T) for theta in thetas]
      result = ray.get(futures)
      ## return result ##handy to debug
      ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

      for j,theta in enumerate(thetas):
          ## if True: ##in case relationships between thetas can be exploited
          ## a dict cannot be used as a key, so we define theta_key, e.g.
          ## theta_key = ((168.0, 72.0), (200.0, 90.0)):
          ## theta_key = tuple(tuple(itm.values()) for itm in theta)
          theta_key = theta ##if theta is not a dict

          ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
          ## CcumIomega__lI = self.run_grid_sample_paths(theta, piName, record)
          ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

          ## Cbarcum_tmp = np.array(CcumIomega__lI).mean() #(SDAM-eq2.9)
          Cbarcum_tmp = np.array(result[j][0]).mean() #(SDAM-eq2.9)
          ## Ctilcum_tmp = np.sum(np.square(np.array(CcumIomega__lI) - Cbarcum_tmp))/(L - 1)
          Ctilcum_tmp = np.sum(np.square(np.array(result[j][0]) - Cbarcum_tmp))/(L - 1)

          Cbarcum[theta_key] = Cbarcum_tmp
          Ctilcum[theta_key] = np.sqrt(Ctilcum_tmp/L)

          ## expCbarcum_tmp = pd.Series(CcumIomega__lI).expanding().mean()
          expCbarcum_tmp = pd.Series(result[j][0]).expanding().mean()
          expCbarcum[theta_key] = expCbarcum_tmp

          ## expCtilcum_tmp = pd.Series(CcumIomega__lI).expanding().std()
          expCtilcum_tmp = pd.Series(result[j][0]).expanding().std()
          expCtilcum[theta_key] = expCtilcum_tmp
          if i%nth == 0: print(f'{i:,}/{(numThetas-1):,}, {Cbarcum[theta_key]:,.0f}, {theta}')
          i += 1
          ##endif
          next_th = result[j][1]
          for e in next_th:
              record.append(e)
      best_theta = max(Cbarcum, key=Cbarcum.get)
      worst_theta = min(Cbarcum, key=Cbarcum.get)

      best_Cbarcum = Cbarcum[best_theta]
      best_Ctilcum = Ctilcum[best_theta]

      worst_Cbarcum = Cbarcum[worst_theta]
      worst_Ctilcum = Ctilcum[worst_theta]

      thetaStar_expCbarcum = expCbarcum[best_theta]
      thetaStar_expCtilcum = expCtilcum[best_theta]
      thetaStar_expCtilcum[0] = 0 ##set NaN to 0

      return \
        thetaStar_expCbarcum, thetaStar_expCtilcum, \
        Cbarcum, Ctilcum, \
        best_theta, worst_theta, \
        best_Cbarcum, worst_Cbarcum, \
        best_Ctilcum, worst_Ctilcum, \
        record

In [ ]:
#| export
## EXAMPLE:
## thetasA: Buy
## thetasA_name: 'thBuy'
## names: ELA
## 1_1: 1 theta sub-vectors, each having 1 theta
## thetas = grid_search_thetas_1_2(thetasBuy 'thBuy', CAR_TYPES)
@patch
def grid_search_thetas_1_1(self:Policy, thetasA, thetasA_name, names):
    thetas = [
    self.build_theta({thetasA_name: {names[0]: thA0}})
    for thA0 in thetasA[names[0]]
    ]
    return thetas

In [ ]:
#| export
## EXAMPLE:
## thetasA: Buy
## thetasA_name: 'thBuy'
## names: ELA, SON
## 1_2: 1 theta sub-vectors, each having 2 thetas
## thetas = grid_search_thetas_1_2(thetasBuy 'thBuy', CAR_TYPES)
@patch
def grid_search_thetas_1_2(self:Policy, thetasA, thetasA_name, names):
    thetas = [
    self.build_theta({thetasA_name: {names[0]: thA0, names[1]: thA1}})
    for thA0 in thetasA[names[0]]
      for thA1 in thetasA[names[1]]
    ]
    return thetas

In [ ]:
## EXAMPLE:
## thetasA: Adj
## thetasA_name: 'thAdj'
## names: ELA, SON
## 1_4: 1 theta sub-vectors, each having 4 thetas
## thetas = grid_search_thetas_1_4(thetasBuy 'thAdj', bNAMES)
@patch
def grid_search_thetas_1_4(self:Policy, thetasA, thetasA_name, names):
    thetas = [
    self.build_theta({thetasA_name: {names[0]: thA0, names[1]: thA1, names[2]: thA2, names[3]: thA3}})
    for thA0 in thetasA[names[0]]
      for thA1 in thetasA[names[1]]
        for thA2 in thetasA[names[2]]
          for thA3 in thetasA[names[3]]
    ]
    return thetas

In [ ]:
## EXAMPLE:
## thetasA: Buy
## thetasB: Max
## thetasA_name: 'thBuy'
## thetasB_name: 'thMax'
## names: ELA
## 2_1: 2 theta sub-vectors, each having 1 theta
## thetas = grid_search_thetas_2_1(thetasBuy, thetasMax, 'thBuy', 'thMax', CAR_TYPES)
@patch
def grid_search_thetas_2_1(self:Policy, thetasA, thetasB, thetasA_name, thetasB_name, names):
    thetas = [
    self.build_theta({thetasA_name: {names[0]: thA0}, thetasB_name: {names[0]: thB0}})
    for thA0 in thetasA[names[0]]
      for thB0 in thetasB[names[0]]
    ]
    return thetas

In [ ]:
## EXAMPLE:
## thetasA: Buy
## thetasB: Max
## thetasA_name: 'thBuy'
## thetasB_name: 'thMax'
## names: ELA, SON
## 2_2: 2 theta sub-vectors, each having 2 thetas
## thetas = grid_search_thetas_4(thetasBuy, thetasMax, 'thBuy', 'thMax', CAR_TYPES)
@patch
def grid_search_thetas_2_2(self:Policy, thetasA, thetasB, thetasA_name, thetasB_name, names):
    thetas = [
    self.build_theta({thetasA_name: {names[0]: thA0, names[1]: thA1}, thetasB_name: {names[0]: thB0, names[1]: thB1}})
    for thA0 in thetasA[names[0]]
      for thA1 in thetasA[names[1]]
        for thB0 in thetasB[names[0]]
          for thB1 in thetasB[names[1]]
    ]
    return thetas

In [ ]:
# from AIShiftScheduler_KE5.model import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()